# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 8 Assignment: Building a Kaggle Submission File**

**Student Name: Wei Lu**

# Assignment Instructions

For this assignment you will use the [**reg-33-data.csv**](http://data.heatonresearch.com/data/t81-558/datasets/reg-33-data.csv) dataset to train a neural network and [**reg-33-eval.csv**](http://data.heatonresearch.com/data/t81-558/datasets/reg-33-eval.csv) to use as test to build a submission (similar to Kaggle).  The preprocessing/feature engineering code used for this assignment will be identical to [Assignmnent 5](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb) and you are encouraged to use your Assignment 5 code as a starting point.  Refer to [Module 8](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class8_kaggle.ipynb) for instructions on producing a Kaggle type submission file.  

The dataframe that you submit should have two columns: *id* and *target*.  The *id* column should matchup with the test data file.  The *target* column is your prediction.  It is unlikely that the mean of *target* will match exactly with mine.

Note, my version generated this warning.  You should be lower than 300, obviously, lower is better!  **Warning: The mean of column target differs from the solution file by 121.90476911730366.**

# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Assignment #8 Sample Code

The following code provides a starting point for this assignment.

In [2]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import zscore

key = "nFpEY2LlkO1YtRFzGfAdnaL2S1FiaWjD7MPdWB8B"
file='/content/drive/My Drive/Colab Notebooks/assignment_WeiLu_class8.ipynb'

# Begin assignment
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/reg-33-data.csv")

# Encode the feature vector
ids = df['id']
df.drop('id',1,inplace=True)


# Normalize all numerics to zscores and all text/categoricals to dummies. Do not normalize the target. 
dict = df.dtypes.to_dict()

for i in df.columns.drop('target'):
    if dict[i] == 'O':                   # Object
        df = pd.concat([df, pd.get_dummies(df[i], prefix=i)], axis=1)
    
    else:                                # numbers
        while pd.isnull(df[i]).sum()!=0: # Dealing with missing values
            med = df[i].median()
            df[i] = df[i].fillna(med)
        df[i] = zscore(df[i])

    df.drop(i, 1, inplace=True)


x_columns = df.columns.drop('target')
x = df[x_columns].values
y = df['target'].values


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,epochs=500)




Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# Generate Kaggle submit file

# Encode feature vector
df_test = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/reg-33-eval.csv",
                      na_values=['NA','?'])

ids = df_test['id']
df_test.drop('id', axis=1, inplace=True)

dict = df_test.dtypes.to_dict()
for i in df_test.columns:
    if dict[i] == 'O':                   # Object
        df_test = pd.concat([df_test, pd.get_dummies(df_test[i], prefix=i)], axis=1)
    
    else:                                # numbers
        while pd.isnull(df_test[i]).sum()!=0: # Dealing with missing values
            med = df_test[i].median()
            df_test[i] = df_test[i].fillna(med)
        df_test[i] = zscore(df_test[i])

    df_test.drop(i, 1, inplace=True)


x_columns = df_test.columns
x = df_test[x_columns].values

# Generate predictions
pred = model.predict(x)

# Create submission data set
df_submit = pd.DataFrame(pred)
df_submit.insert(0,'id',ids)
df_submit.columns = ['id','target']

submit(source_file=file,data=df_submit,key=key,no=8)

Success: Submitted Assignment #8 for luwei:
You have submitted this assignment 6 times. (this is fine)

